# SECTION 1: IMPORTS & SETUP

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("LAST MILE CONNECT - GEOGRAPHIC CLUSTERING")
print("="*80)

DATA_PATH = Path("../data/processed")
EXTERNAL_PATH = Path("../data/external")
VIZ_PATH = Path("../docs/visualizations")
VIZ_PATH.mkdir(parents=True, exist_ok=True)

LAST MILE CONNECT - GEOGRAPHIC CLUSTERING


# SECTION 2: DATA LOADING

In [21]:
print("\n Loading processed data...")

# Load gap analysis results
gap_df = pd.read_csv(DATA_PATH / "district_gap_analysis.csv")
print(f"✅ District gap analysis: {len(gap_df)} districts")

# Load pincode-level enrolment
try:
    pincode_df = pd.read_csv(DATA_PATH / "pincode_enrolment.csv")
    print(f"✅ Pincode data: {len(pincode_df):,} pincodes")
    HAS_PINCODE_DATA = True
except FileNotFoundError:
    print("⚠️  Pincode data not found - using district-level aggregation")
    HAS_PINCODE_DATA = False

# Load pincode geolocation data
try:
    pincode_geo = pd.read_csv(EXTERNAL_PATH / "india_pincodes.csv")
    # Standardize column names
    pincode_geo.columns = [col.lower().strip() for col in pincode_geo.columns]
    
    # Ensure we have required columns
    required_cols = ['pincode', 'latitude', 'longitude']
    if all(col in pincode_geo.columns for col in required_cols):
        print(f"✅ Pincode geolocation: {len(pincode_geo):,} pincodes")
        HAS_GEO_DATA = True
    else:
        print(f"⚠️  Pincode file missing required columns: {required_cols}")
        HAS_GEO_DATA = False
except FileNotFoundError:
    print("⚠️  Pincode geolocation data not found")
    HAS_GEO_DATA = False


 Loading processed data...
✅ District gap analysis: 1045 districts
✅ Pincode data: 28,913 pincodes
✅ Pincode geolocation: 165,627 pincodes


# SECTION 3: PREPARE CLUSTERING DATA

In [22]:
print("\n Preparing data for clustering...")

if HAS_PINCODE_DATA and HAS_GEO_DATA:
    print("✅ Using PINCODE-LEVEL clustering (optimal)")
    
    # Merge pincode enrolment with gap analysis
    clustering_data = pincode_df.merge(
        gap_df[['state', 'district', 'unreached_population', 
                'coverage_rate', 'priority_level']],
        on=['state', 'district'],
        how='left'
    )
    
    # Merge with geolocation
    clustering_data = clustering_data.merge(
        pincode_geo[['pincode', 'latitude', 'longitude']],
        on='pincode',
        how='left'
    )
    
    # Remove pincodes without coordinates
    clustering_data = clustering_data.dropna(subset=['latitude', 'longitude'])
    
    # Estimate unreached per pincode (proportional distribution)
    district_totals = clustering_data.groupby('district')['total_enrolment'].sum().to_dict()
    clustering_data['enrolment_share'] = clustering_data.apply(
        lambda x: x['total_enrolment'] / district_totals.get(x['district'], 1), 
        axis=1
    )
    clustering_data['estimated_unreached'] = (
        clustering_data['unreached_population'] * clustering_data['enrolment_share']
    ).round(0).astype(int)
    
    print(f"   Pincodes with coordinates: {len(clustering_data):,}")
    
else:
    print("⚠️  Using DISTRICT-LEVEL clustering (suboptimal)")
    print("   RECOMMENDATION: Get pincode geolocation data for better results")
    
    # Create synthetic coordinates at district centroids
    # This is a FALLBACK - not ideal for production
    clustering_data = gap_df.copy()
    
    # Estimate coordinates based on state centroids
    state_centroids = {
        'Andhra Pradesh': (15.9129, 79.7400),
        'Bihar': (25.0961, 85.3131),
        'Gujarat': (22.2587, 71.1924),
        'Karnataka': (15.3173, 75.7139),
        'Kerala': (10.8505, 76.2711),
        'Maharashtra': (19.7515, 75.7139),
        'Tamil Nadu': (11.1271, 78.6569),
        'Uttar Pradesh': (26.8467, 80.9462),
        'West Bengal': (22.9868, 87.8550),
        # Add more states as needed
    }
    
    np.random.seed(42)
    def estimate_coords(row):
        centroid = state_centroids.get(row['state'], (20.0, 77.0))
        # Add random jitter (±1 degree)
        lat = centroid[0] + np.random.uniform(-1, 1)
        lon = centroid[1] + np.random.uniform(-1, 1)
        return pd.Series([lat, lon])
    
    clustering_data[['latitude', 'longitude']] = clustering_data.apply(
        estimate_coords, axis=1
    )
    clustering_data['estimated_unreached'] = clustering_data['unreached_population']


 Preparing data for clustering...
✅ Using PINCODE-LEVEL clustering (optimal)
   Pincodes with coordinates: 236,207


# SECTION 4: FILTER HIGH-PRIORITY AREAS

In [23]:
print("\n Filtering high-priority regions...")

# Focus on CRITICAL and HIGH priority areas
priority_data = clustering_data[
    clustering_data['priority_level'].isin(['CRITICAL', 'HIGH'])
].copy()

print(f"   High-priority areas: {len(priority_data):,}")
print(f"   Unreached population: {priority_data['estimated_unreached'].sum():,}")

# Further filter: locations with significant unreached population
MIN_UNREACHED_THRESHOLD = 100  # At least 100 unreached citizens
priority_data = priority_data[
    priority_data['estimated_unreached'] >= MIN_UNREACHED_THRESHOLD
].copy()

print(f"   After filtering (≥{MIN_UNREACHED_THRESHOLD} unreached): {len(priority_data):,}")


 Filtering high-priority regions...
   High-priority areas: 236,207
   Unreached population: 1,614,106,760
   After filtering (≥100 unreached): 213,422


# SECTION 5: DETERMINE OPTIMAL NUMBER OF CLUSTERS (CAMPS)

In [24]:
print("\n" + "="*80)
print(" DETERMINING OPTIMAL NUMBER OF MOBILE CAMPS")
print("="*80)

# Prepare features
features = priority_data[['latitude', 'longitude']].copy()

# Force numeric conversion
features['latitude'] = pd.to_numeric(features['latitude'], errors='coerce')
features['longitude'] = pd.to_numeric(features['longitude'], errors='coerce')

# Remove invalid rows
valid_idx = features.dropna().index
features = features.loc[valid_idx]

# Weight by unreached population (sqrt to limit outliers)
weights = np.sqrt(
    priority_data.loc[valid_idx, 'estimated_unreached']
    .astype(float)
    .values
)

# Apply weighting
features_weighted = features.values * weights[:, np.newaxis]

# Standardize
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_weighted)

# Elbow method
from sklearn.cluster import KMeans

print("\n🔍 Running elbow method...")

inertias = []
K_range = range(50, 351, 50)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(features_scaled)
    inertias.append(kmeans.inertia_)

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(K_range, inertias, marker='o', linewidth=2, markersize=8)
ax.set_xlabel('Number of Camps (K)')
ax.set_ylabel('Inertia')
ax.set_title('Elbow Method - Optimal Number of Mobile Camps', fontweight='bold')
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(VIZ_PATH / 'elbow_method.png', dpi=300, bbox_inches='tight')
plt.close()

print("✅ Saved: elbow_method.png")

OPTIMAL_CAMPS = 200
print(f"\n🎯 Selected number of mobile camps: {OPTIMAL_CAMPS}")



 DETERMINING OPTIMAL NUMBER OF MOBILE CAMPS

🔍 Running elbow method...
✅ Saved: elbow_method.png

🎯 Selected number of mobile camps: 200


# SECTION 6: PERFORM K-MEANS CLUSTERING

In [25]:
print("\n Performing K-Means clustering...")

# Fit final model
kmeans_final = KMeans(
    n_clusters=OPTIMAL_CAMPS,
    random_state=42,
    n_init=20
)

labels = kmeans_final.fit_predict(features_scaled)

# Assign clusters safely
priority_data['camp_cluster'] = np.nan
priority_data.loc[valid_idx, 'camp_cluster'] = labels

# Get cluster centers (standardized → original space)
camp_centers = scaler.inverse_transform(
    kmeans_final.cluster_centers_
)

# Create camp locations dataframe
camps_df = pd.DataFrame(
    camp_centers,
    columns=['latitude', 'longitude']
)
camps_df['camp_id'] = range(1, len(camps_df) + 1)

print(f"✅ {OPTIMAL_CAMPS} camp locations identified")



 Performing K-Means clustering...
✅ 200 camp locations identified


# SECTION 6: ASSIGN NEAREST DISTRICT TO EACH CAMP

In [26]:
print("\n Assigning districts to camps...")

# -----------------------------------------------------------
# STEP 1: Ensure coordinates are numeric
# -----------------------------------------------------------
priority_data['latitude'] = pd.to_numeric(priority_data['latitude'], errors='coerce')
priority_data['longitude'] = pd.to_numeric(priority_data['longitude'], errors='coerce')

camps_df['latitude'] = pd.to_numeric(camps_df['latitude'], errors='coerce')
camps_df['longitude'] = pd.to_numeric(camps_df['longitude'], errors='coerce')

# Drop invalid rows (safety)
priority_valid = priority_data.dropna(subset=['latitude', 'longitude']).copy()

# Extract arrays for fast computation
priority_coords = priority_valid[['latitude', 'longitude']].values

# -----------------------------------------------------------
# STEP 2: Find nearest district for each camp 
# -----------------------------------------------------------
camp_states = []
camp_districts = []

for camp_lat, camp_lon in camps_df[['latitude', 'longitude']].values:
    # Vectorized distance computation
    distances = np.sqrt(
        (priority_coords[:, 0] - camp_lat) ** 2 +
        (priority_coords[:, 1] - camp_lon) ** 2
    )

    nearest_idx = distances.argmin()
    nearest_row = priority_valid.iloc[nearest_idx]

    camp_states.append(nearest_row['state'])
    camp_districts.append(nearest_row['district'])

camps_df['state'] = camp_states
camps_df['district'] = camp_districts

# -----------------------------------------------------------
# STEP 3: Assign nearest pincode 
# -----------------------------------------------------------
if HAS_PINCODE_DATA and 'pincode' in priority_valid.columns:
    camp_pincodes = []

    for camp_lat, camp_lon in camps_df[['latitude', 'longitude']].values:
        distances = np.sqrt(
            (priority_coords[:, 0] - camp_lat) ** 2 +
            (priority_coords[:, 1] - camp_lon) ** 2
        )
        nearest_idx = distances.argmin()
        camp_pincodes.append(priority_valid.iloc[nearest_idx]['pincode'])

    camps_df['nearest_pincode'] = camp_pincodes

print("✅ District and pincode assignments completed successfully")



 Assigning districts to camps...
✅ District and pincode assignments completed successfully


# SECTION 8: CALCULATE COVERAGE STATISTICS PER CAMP

In [27]:
print("\n Calculating coverage statistics...")

# Calculate coverage per camp
camp_stats = priority_data.groupby('camp_cluster').agg({
    'estimated_unreached': 'sum',
    'state': 'first'  # Just to get count
}).reset_index()

# Count number of locations assigned to each camp
location_counts = priority_data.groupby('camp_cluster').size()
camp_stats['num_locations'] = camp_stats['camp_cluster'].map(location_counts)

camp_stats.columns = ['camp_cluster', 'coverage_population', 'state', 'num_locations']
camp_stats = camp_stats.drop('state', axis=1)

# Merge with camps dataframe
camps_df = camps_df.merge(
    camp_stats,
    left_index=True,
    right_on='camp_cluster',
    how='left'
).drop('camp_cluster', axis=1)

print(f"\n📊 Camp Statistics:")
print(f"   Total camps: {len(camps_df)}")
print(f"   Total coverage: {camps_df['coverage_population'].sum():,.0f} unreached citizens")
print(f"   Avg per camp: {camps_df['coverage_population'].mean():,.0f} citizens")
print(f"   Median per camp: {camps_df['coverage_population'].median():,.0f} citizens")



 Calculating coverage statistics...

📊 Camp Statistics:
   Total camps: 200
   Total coverage: 1,613,232,575 unreached citizens
   Avg per camp: 8,066,163 citizens
   Median per camp: 5,766,356 citizens


# SECTION 9: PRIORITY RANKING OF CAMPS

In [28]:
print("\n Ranking camps by priority...")

# Sort camps by coverage population (descending)
camps_df = camps_df.sort_values('coverage_population', ascending=False).reset_index(drop=True)
camps_df['priority_rank'] = range(1, len(camps_df) + 1)

# Classify camp priority
def classify_camp_priority(rank, total_camps):
    if rank <= total_camps * 0.2:  # Top 20%
        return 'CRITICAL'
    elif rank <= total_camps * 0.5:  # Top 50%
        return 'HIGH'
    elif rank <= total_camps * 0.8:  # Top 80%
        return 'MEDIUM'
    else:
        return 'LOW'

camps_df['camp_priority'] = camps_df['priority_rank'].apply(
    lambda x: classify_camp_priority(x, len(camps_df))
)

print(f"\n📊 Camp Priority Distribution:")
print(camps_df['camp_priority'].value_counts())

print(f"\n🎯 TOP 10 PRIORITY CAMPS:")
print(camps_df.head(10)[
    ['camp_id', 'state', 'district', 'coverage_population', 
     'num_locations', 'camp_priority', 'latitude', 'longitude']
].to_string(index=False))


 Ranking camps by priority...

📊 Camp Priority Distribution:
camp_priority
HIGH        60
MEDIUM      60
CRITICAL    40
LOW         40
Name: count, dtype: int64

🎯 TOP 10 PRIORITY CAMPS:
 camp_id     state  district  coverage_population  num_locations camp_priority    latitude    longitude
      71 Jharkhand   Giridih             26091173           2639      CRITICAL 2335.304734  7746.997903
     134 Jharkhand   Giridih             25653397           2748      CRITICAL 2273.528065  7859.900032
      95 Jharkhand   Giridih             24351752            957      CRITICAL 3993.519850 12176.244716
     135 Jharkhand   Giridih             24181800           1490      CRITICAL 3157.721381 10077.548745
     103 Jharkhand   Giridih             23936412           1290      CRITICAL 3354.368902 10691.412801
     136 Jharkhand   Giridih             23046933           1696      CRITICAL 2838.154143  9289.357389
      96 Jharkhand   Giridih             22360193           1015      CRITICAL 3600.

# SECTION 10: COST ESTIMATION

In [29]:
print("\n Estimating deployment costs...")

# Cost assumptions (in INR)
COST_PER_CAMP_SETUP = 150000  # ₹1.5 lakh setup
COST_PER_CAMP_OPERATION_DAY = 25000  # ₹25k per day
ENROLLMENT_CAPACITY_PER_DAY = 500  # 500 enrollments/day
COST_PER_ENROLLMENT = 50  # ₹50 materials/personnel per enrollment

# Calculate costs
camps_df['estimated_days'] = np.ceil(
    camps_df['coverage_population'] / ENROLLMENT_CAPACITY_PER_DAY
).astype(int)

camps_df['setup_cost'] = COST_PER_CAMP_SETUP
camps_df['operational_cost'] = camps_df['estimated_days'] * COST_PER_CAMP_OPERATION_DAY
camps_df['enrollment_cost'] = camps_df['coverage_population'] * COST_PER_ENROLLMENT
camps_df['total_cost'] = (
    camps_df['setup_cost'] + 
    camps_df['operational_cost'] + 
    camps_df['enrollment_cost']
)

total_budget = camps_df['total_cost'].sum()

print(f"\n💰 BUDGET ESTIMATION:")
print(f"   Total camps: {len(camps_df)}")
print(f"   Total coverage: {camps_df['coverage_population'].sum():,.0f} citizens")
print(f"   Total budget: ₹{total_budget:,.0f} ({total_budget/10**7:.2f} Crores)")
print(f"   Cost per enrollment: ₹{total_budget/camps_df['coverage_population'].sum():.2f}")
print(f"   Avg days per camp: {camps_df['estimated_days'].mean():.0f}")


 Estimating deployment costs...

💰 BUDGET ESTIMATION:
   Total camps: 200
   Total coverage: 1,613,232,575 citizens
   Total budget: ₹161,355,603,750 (16135.56 Crores)
   Cost per enrollment: ₹100.02
   Avg days per camp: 16133


# SECTION 10: VISUALIZATIONS

In [30]:
print("\n Creating visualizations...")

# Visualization 1: Geographic scatter with camps
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Plot all priority locations
axes[0].scatter(
    priority_data['longitude'], 
    priority_data['latitude'],
    c=priority_data['estimated_unreached'], 
    cmap='YlOrRd',
    s=10, alpha=0.5, label='Priority Locations'
)

# Plot camp locations
axes[0].scatter(
    camps_df['longitude'], 
    camps_df['latitude'],
    c='blue', s=100, marker='^', 
    edgecolor='black', linewidth=1.5,
    label=f'Mobile Camps (n={len(camps_df)})', zorder=5
)

axes[0].set_xlabel('Longitude', fontsize=12)
axes[0].set_ylabel('Latitude', fontsize=12)
axes[0].set_title('Proposed Mobile Camp Locations', fontsize=14, fontweight='bold')
axes[0].legend(loc='best')
axes[0].grid(alpha=0.3)

# Plot top 50 priority camps
top_50 = camps_df.head(50)
scatter = axes[1].scatter(
    top_50['longitude'], 
    top_50['latitude'],
    c=top_50['coverage_population'], 
    cmap='RdYlGn_r',
    s=top_50['coverage_population']/100, 
    alpha=0.7,
    edgecolor='black', linewidth=0.5
)
axes[1].set_xlabel('Longitude', fontsize=12)
axes[1].set_ylabel('Latitude', fontsize=12)
axes[1].set_title('Top 50 Priority Camps (Size = Unreached Population)', 
                   fontsize=14, fontweight='bold')
plt.colorbar(scatter, ax=axes[1], label='Unreached Population')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(VIZ_PATH / 'mobile_camp_locations.png', dpi=300, bbox_inches='tight')
print("✅ Saved: mobile_camp_locations.png")
plt.close()

# Visualization 2: Coverage distribution
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Coverage population histogram
axes[0, 0].hist(camps_df['coverage_population'], bins=30, color='steelblue', edgecolor='black')
axes[0, 0].set_xlabel('Coverage Population')
axes[0, 0].set_ylabel('Number of Camps')
axes[0, 0].set_title('Distribution of Camp Coverage')
axes[0, 0].grid(alpha=0.3)

# Priority distribution
priority_counts = camps_df['camp_priority'].value_counts()
axes[0, 1].bar(priority_counts.index, priority_counts.values, color=['#d32f2f', '#f57c00', '#fbc02d', '#388e3c'])
axes[0, 1].set_xlabel('Priority Level')
axes[0, 1].set_ylabel('Number of Camps')
axes[0, 1].set_title('Camp Priority Distribution')
axes[0, 1].grid(axis='y', alpha=0.3)

# Cost distribution
axes[1, 0].hist(camps_df['total_cost']/10**5, bins=30, color='coral', edgecolor='black')
axes[1, 0].set_xlabel('Cost per Camp (Lakhs)')
axes[1, 0].set_ylabel('Number of Camps')
axes[1, 0].set_title('Distribution of Camp Costs')
axes[1, 0].grid(alpha=0.3)

# Days vs Coverage scatter
axes[1, 1].scatter(camps_df['coverage_population'], camps_df['estimated_days'], 
                    alpha=0.6, s=30, color='seagreen')
axes[1, 1].set_xlabel('Coverage Population')
axes[1, 1].set_ylabel('Estimated Days')
axes[1, 1].set_title('Coverage vs Operational Days')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(VIZ_PATH / 'camp_statistics.png', dpi=300, bbox_inches='tight')
print("✅ Saved: camp_statistics.png")
plt.close()


 Creating visualizations...
✅ Saved: mobile_camp_locations.png
✅ Saved: camp_statistics.png


# SECTION 11: SAVE RESULTS

In [31]:
print("\n Saving clustering results...")

camps_df.to_csv(DATA_PATH / "mobile_camp_locations.csv", index=False)
print("✅ Saved: mobile_camp_locations.csv")

if HAS_PINCODE_DATA:

    # Ensure camp_id exists
    if 'camp_id' not in camps_df.columns:
        camps_df['camp_id'] = camps_df.index + 1

    # Optional: add camp priority if not already present
    if 'camp_priority' not in camps_df.columns:
        camps_df['camp_priority'] = camps_df['camp_id']

    assignments = (
        priority_data[[
            'state', 'district', 'pincode',
            'latitude', 'longitude',
            'estimated_unreached', 'camp_cluster'
        ]]
        .merge(
            camps_df[['camp_id', 'camp_priority']],
            left_on='camp_cluster',   # cluster label (0…K-1)
            right_index=True,         # camp index (0…K-1)
            how='left'
        )
    )

    assignments.to_csv(
        DATA_PATH / "pincode_camp_assignments.csv",
        index=False
    )

    print("✅ Saved: pincode_camp_assignments.csv")



 Saving clustering results...
✅ Saved: mobile_camp_locations.csv
✅ Saved: pincode_camp_assignments.csv


# FINAL SUMMARY

In [32]:
print("\n" + "="*80)
print("✨ GEOGRAPHIC CLUSTERING COMPLETED!")
print("="*80)

print(f"\n📊 KEY OUTPUTS:")
print(f"   • {len(camps_df)} mobile camp locations identified")
print(f"   • GPS coordinates for each camp")
print(f"   • Estimated budget: ₹{total_budget/10**7:.2f} Crores")
print(f"   • Expected coverage: {camps_df['coverage_population'].sum():,.0f} citizens")

print(f"\n📁 FILES CREATED:")
print(f"   • mobile_camp_locations.csv - Camp GPS coordinates and details")
if HAS_PINCODE_DATA:
    print(f"   • pincode_camp_assignments.csv - Pincode-to-camp mapping")
print(f"   • 3 visualization PNG files")


✨ GEOGRAPHIC CLUSTERING COMPLETED!

📊 KEY OUTPUTS:
   • 200 mobile camp locations identified
   • GPS coordinates for each camp
   • Estimated budget: ₹16135.56 Crores
   • Expected coverage: 1,613,232,575 citizens

📁 FILES CREATED:
   • mobile_camp_locations.csv - Camp GPS coordinates and details
   • pincode_camp_assignments.csv - Pincode-to-camp mapping
   • 3 visualization PNG files
